# Langchain: Prompts, Output Parsers and Memories

<a href="https://colab.research.google.com/github/amadeus-art/azure-openai-coding-dojo/blob/main/azure_openai_langchain_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial we are going to introduce langchain and its base components. We will specifically cover how to format prompts and how to use output parsers to extract information from the output of a model and post-process it.

## Install dependencies

In [ ]:
!pip install openai python-dotenv langchain -q

## Environment Setup
Before executing the following cells, make sure to set the `AZURE_OPENAI_KEY` and `AZURE_OPENAI_ENDPOINT` variables in the `.env` file or export them.

<br/>
<img src="assets/keys_endpoint.png" width="800"/>

In [2]:
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

api_key = os.getenv('AZURE_OPENAI_API_KEY')
if api_key:
    print("Environment variable 'AZURE_OPENAI_API_KEY' is set")
else:
    print("Environment variable 'AZRUE_OPENAI_API_KEY' is not set")

Environment variable 'AZURE_OPENAI_API_KEY' is set


## Prompts and Output Parsers

In [3]:
from langchain_openai import AzureChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0
chat = AzureChatOpenAI(
    azure_deployment=os.getenv("MODEL_DEPLOYMENT_NAME"),
    openai_api_version='2024-02-01',
    temperature=0
)

In [4]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [5]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [6]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [7]:
customer_style = """American English \
in a calm and respectful tone
"""

In [8]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [9]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [10]:
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")]

In [11]:
customer_response = chat.invoke(customer_messages)

In [12]:
customer_response.content

"I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?"

## Output Parsers
Let's start with defining how we would like the LLM output to look like:


In [13]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [14]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [16]:
messages = prompt_template.format_messages(text=customer_review)
response = chat.invoke(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [17]:
type(response.content) # this is not a dictionary!

str

### Parse the LLM output string into a Python dictionary using Parsers

In [18]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [19]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [20]:
format_instructions = output_parser.get_format_instructions()

In [21]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [22]:
type(output_dict)

dict